In [1]:
import clang.cindex
from collections import namedtuple
from typing import *
import re

## Utils

In [2]:
class src_list:          
    def __init__(self):
        self.cur_indent = 0;
        self.src = ""
        
    def __iadd__(self, l:str):
        self.src += " "*self.cur_indent + l + "\n";
        return self
    
    def __rshift__(self, ind):   self.cur_indent += ind;
    def __lshift__(self, ind): self.cur_indent -= ind;

    class indent_ctx:
        def __init__(self, slist, ind:int):
            self.src_list = slist
            self.indent = ind
        def __enter__(self):
            self.src_list >> self.indent
        def __exit__(self, *args):
            self.src_list << self.indent
        
    def indent(self, ind:int=4):
        return src_list.indent_ctx(self, ind)
    
    def get(self):
        return self.src

In [3]:
sl = src_list()
sl += "class A {"
with sl.indent(2):
    sl += "public:"
with sl.indent():
    sl += "int a() { return pmember_a; }"
with sl.indent(2):
    sl += "private:"
    with sl.indent(2):
        sl += "int pmember_a;"
sl += "}"
print(sl.get())

class A {
  public:
    int a() { return pmember_a; }
  private:
    int pmember_a;
}



In [4]:
# export
def enumerate_nodes(node, node_kind:clang.cindex.CursorKind):
    def find_nodes(node, node_kind:clang.cindex.CursorKind):
        nodes = []
        if node.kind == node_kind:
            nodes.append(node)
        for c in node.get_children():
            nodes += find_nodes(c, node_kind)
        return nodes
    
    nodes = find_nodes(node, node_kind)
    node_map = {node.spelling: node for node in nodes}
    return node_map

In [5]:
# export
def split_uppercase(s):
    return [a for a in re.split(r'([A-Z][a-z]*)', s) if a]

def structure_type_name(type_name:str):
    parts = split_uppercase(type_name)[1:]
    stype_parts = ['vk','structure','type']+parts
    return "_".join([s.upper() for s in stype_parts])

In [6]:
%pdb on

Automatic pdb calling has been turned ON


In [7]:
def camel_to_snake(s:str, remove_prefixes=False):
    parts = split_uppercase(s)
    if remove_prefixes:
        while parts[0] in ['p', 'pp']:
            parts = parts[1:]
    # merge extension suffix
    start_suffix_merge = len(parts)
    while len(parts[start_suffix_merge-1]) == 1: 
        start_suffix_merge -= 1
    if start_suffix_merge < len(parts):
        parts[start_suffix_merge:] = [''.join(parts[start_suffix_merge:])]
    return "_".join([s.lower() for s in parts])

In [8]:
camel_to_snake('vkCmdBeginConditionalRenderingEXT'), camel_to_snake("queueFamilyIndexCount"), camel_to_snake("pNext", True)

('vk_cmd_begin_conditional_rendering_ext', 'queue_family_index_count', 'next')

In [9]:
# export
member = namedtuple("Member", ["type", "name"])
def list_members(node)->List[member]:
    members = []
    for n in node.get_children():
        members.append(member(n.type.spelling, n.spelling))
    return members

### Map between cory and Vulkan types automatically
For example, in the automatically generated interfaces we want to use `glm::uvec3` instead of `VkExtent3D`. To define the mapping between the two types, we define bidirectional mappers that encapsulate the code fragment to insert in order to map a variable of one type to the other.

In [10]:
class type_mapper:
    def __init__(self, vk_type, cory_type, v2c, c2v):
        self.vk_type = vk_type
        self.cory_type = cory_type
        self.c2v = c2v
        self.v2c = v2c
        
    def cory_to_vk(self, var_name:str):
        return self.c2v.format_map({'var':var_name})
    def vk_to_cory(self, var_name:str):
        return self.v2c.format_map({'var':var_name})

In [11]:
cory_type_mappers = {}
vulkan_type_mappers = {}
def register_type_mapper(tm:type_mapper):
    cory_type_mappers[tm.cory_type] = tm
    vulkan_type_mappers[tm.vk_type] = tm
def find_vk_mapper(vk_type):
    if vk_type in vulkan_type_mappers:
        return vulkan_type_mappers[vk_type]
    return type_mapper(vk_type, vk_type, "{var}", "{var}")
def find_cory_mapper(cory_type):
    if cory_type in cory_type_mappers:
        return cory_type_mappers[cory_type]
    return type_mapper(cory_type, cory_type, "{var}", "{var}")

In [12]:
register_type_mapper(
    type_mapper("VkExtent3D", "glm::uvec3", 
                 "glm::uvec3{{ {var}.width, {var}.height, {var}.depth }}",
                 "VkExtent3D{{ {var}.x, {var}.y, {var}.z }}"
                ))

In [13]:
register_type_mapper(
    type_mapper("const char *const *", "std::vector<const char*>", 
                 "undefined",
                 "{var}.data()"
                ))

## Parsing vulkan.h

In [14]:
index = clang.cindex.Index.create()

In [17]:
VULKAN_HEADER_FILE = R"C:\Users\j\.conan\data\vulkan-headers\1.2.154.0\_\_\package\5ab84d6acfe1f23c4fae0ab88f26e3a396351ac9\include\vulkan\vulkan_core.h"
#VULKAN_HEADER_FILE = R"ThirdParty/include/flextVk.h"

In [18]:
translation_unit = index.parse(VULKAN_HEADER_FILE)

In [19]:
translation_unit

In [20]:
vk_structs = enumerate_nodes(translation_unit.cursor, clang.cindex.CursorKind.STRUCT_DECL)
vk_enums = enumerate_nodes(translation_unit.cursor, clang.cindex.CursorKind.ENUM_DECL)
vk_funcs = enumerate_nodes(translation_unit.cursor, clang.cindex.CursorKind.FUNCTION_DECL)

In [21]:
vk_create_info_names = [k for k in vk_structs.keys() if k.find("CreateInfo") >=0]

In [22]:
ici_node = vk_structs["VkImageCreateInfo"]

In [23]:
list_members(ici_node)

[Member(type='VkStructureType', name='sType'),
 Member(type='const void *', name='pNext'),
 Member(type='VkImageCreateFlags', name='flags'),
 Member(type='VkImageType', name='imageType'),
 Member(type='VkFormat', name='format'),
 Member(type='VkExtent3D', name='extent'),
 Member(type='uint32_t', name='mipLevels'),
 Member(type='uint32_t', name='arrayLayers'),
 Member(type='VkSampleCountFlagBits', name='samples'),
 Member(type='VkImageTiling', name='tiling'),
 Member(type='VkImageUsageFlags', name='usage'),
 Member(type='VkSharingMode', name='sharingMode'),
 Member(type='uint32_t', name='queueFamilyIndexCount'),
 Member(type='const uint32_t *', name='pQueueFamilyIndices'),
 Member(type='VkImageLayout', name='initialLayout')]

## Creating a builder for the `*CreateInfo` structs

In [24]:
member_defaults = {
    member('VkImageFormat', 'format'): "VK_IMAGE_FORMAT_UNDEFINED",
    member('VkExtent2D', 'extent'): "{1, 1}",
    member('VkExtent3D', 'extent'): "{1, 1, 1}",
    member('uint32_t', 'mipLevels'): "1",
    member('uint32_t', 'arrayLayers'): "1",
    member('VkSampleCountFlagBits', 'samples'): "VK_SAMPLE_COUNT_1_BIT",
    member('VkComponentMapping', 'components'): R"""VkComponentMapping{.r = VK_COMPONENT_SWIZZLE_R,
   .g = VK_COMPONENT_SWIZZLE_G,
   .b = VK_COMPONENT_SWIZZLE_B,
   .a = VK_COMPONENT_SWIZZLE_A}"""
}

In [25]:
class builder_builder:
    def __init__(self, built_class, node, with_usage = False):
        self.name = f"{built_class}_builder"
        self.built_class = built_class
        self.node = node
        self.struct_name = self.node.spelling
        self.struct_members = list_members(self.node)
        self.with_usage = with_usage
    
    def process_members(self, src):
        for member in self.struct_members:
            if member.name == 'sType': continue
            setter_name = camel_to_snake(member.name, remove_prefixes=True)
            mapper = find_vk_mapper(member.type)
            src += f"{self.name}& {setter_name}({mapper.cory_type} {member.name}) noexcept {{"
            with src.indent():
                src += f"info_.{member.name} = {mapper.cory_to_vk(member.name)};"
                src += "return *this;"
            src += "}\n"          
    
    def usage_setter(self, src):
        if not self.with_usage: return
        src += f"{self.name}& memory_usage(device_memory_usage usage) noexcept {{"
        with src.indent():
            src += "usage_ = usage;"
            src += "return *this;"
        src += "}"
        
    def name_setter(self, src):
        src += f"{self.name}& name(std::string_view name) noexcept {{"
        with src.indent():
            src += "name_ = name;"
            src += "return *this;"
        src += "}"
    
    def constructor(self, src):
        src += f"{self.name}(graphics_context& context): ctx_{{context}} {{}}\n"

    def conversion_operator(self, src):
        src += f"[[nodiscard]] operator {self.built_class}() {{"
        with src.indent():
            src += f"return ctx_.create_{self.built_class}(*this);"
        src += "}\n"
        
    def create_function(self, src):
        src += f"[[nodiscard]] {self.built_class} create() {{";
        with src.indent():
            src += f"return {self.built_class}(*this);"
        src += "}"
    
    def default_values(self, src):
        src += f".sType = {structure_type_name(self.struct_name)},"
        for m in self.struct_members:
            if m in member_defaults:
                src += f".{m.name} = {member_defaults[m]},"
        
    def build(self):
        src = src_list()
        src += f"class {self.name} {{"
        with src.indent(2):
            src += "public:"
            
        with src.indent():
            src += f"friend class {self.built_class};"
            self.constructor(src)
            self.process_members(src)
            self.usage_setter(src)
            self.name_setter(src)
            #self.conversion_operator(src)
            self.create_function(src)
        
        with src.indent(2):
            src += "private:"
        with src.indent(4):
            src += "graphics_context& ctx_;"
            src += f"{self.struct_name} info_{{"
            with src.indent():
                self.default_values(src)
            src += "};"
            src += "std::string_view name_;"
            if self.with_usage:
                src += "device_memory_usage usage_{device_memory_usage::eGpuOnly};"
        src += "};"
        
        return src.src

In [53]:
#b = builder_builder("image", vk_structs["VkImageCreateInfo"], with_usage=True)
#print(b.build())

In [52]:
#print(builder_builder("image_view", vk_structs["VkImageViewCreateInfo"]).build())

In [50]:
#print(builder_builder("buffer", vk_structs["VkBufferCreateInfo"], with_usage=True).build())

In [42]:
#print(builder_builder("instance", vk_structs["VkInstanceCreateInfo"]).build())

In [43]:
#print(builder_builder("queue", vk_structs["VkDeviceQueueCreateInfo"]).build())
#print(builder_builder("device", vk_structs["VkDeviceCreateInfo"]).build())

In [44]:
#print(builder_builder("swapchain", vk_structs["VkSwapchainCreateInfoKHR"]).build())

In [45]:
#print(builder_builder("image_view", vk_structs["VkImageViewCreateInfo"]).build())

In [46]:
#print(builder_builder("debug_utils_messenger", vk_structs["VkDebugUtilsMessengerCreateInfoEXT"]).build())

In [47]:
#print(builder_builder("command_pool", vk_structs["VkCommandPoolCreateInfo"]).build())

In [48]:
#print(builder_builder("submit_info", vk_structs["VkSubmitInfo"]).build())

In [49]:
#print(builder_builder("command_buffer_allocate_info", vk_structs["VkCommandBufferAllocateInfo"]).build())

In [55]:
#print(builder_builder("render_pass", vk_structs["VkRenderPassCreateInfo"]).build())

In [58]:
#print(builder_builder("attachment_description", vk_structs["VkAttachmentDescription"]).build())

In [59]:
print(builder_builder("subpass_description", vk_structs["VkSubpassDescription"]).build())

class subpass_description_builder {
  public:
    friend class subpass_description;
    subpass_description_builder(graphics_context& context): ctx_{context} {}

    subpass_description_builder& flags(VkSubpassDescriptionFlags flags) noexcept {
        info_.flags = flags;
        return *this;
    }

    subpass_description_builder& pipeline_bind_point(VkPipelineBindPoint pipelineBindPoint) noexcept {
        info_.pipelineBindPoint = pipelineBindPoint;
        return *this;
    }

    subpass_description_builder& input_attachment_count(uint32_t inputAttachmentCount) noexcept {
        info_.inputAttachmentCount = inputAttachmentCount;
        return *this;
    }

    subpass_description_builder& input_attachments(const VkAttachmentReference * pInputAttachments) noexcept {
        info_.pInputAttachments = pInputAttachments;
        return *this;
    }

    subpass_description_builder& color_attachment_count(uint32_t colorAttachmentCount) noexcept {
        info_.colorAttachmentCount 

## Generate enum to_string serializers

In [37]:
enum_name = namedtuple("EnumName", ["type", "name", "value"])

In [38]:
def list_enum_names(node):
    members = []
    for n in node.get_children():
        members.append(enum_name(n.type.spelling, n.spelling, n.enum_value))
    return members

In [39]:
list_enum_names(vk_enums['VkImageTiling'])

[EnumName(type='int', name='VK_IMAGE_TILING_OPTIMAL', value=0),
 EnumName(type='int', name='VK_IMAGE_TILING_LINEAR', value=1),
 EnumName(type='int', name='VK_IMAGE_TILING_DRM_FORMAT_MODIFIER_EXT', value=1000158000),
 EnumName(type='int', name='VK_IMAGE_TILING_MAX_ENUM', value=2147483647)]

In [40]:
def make_to_string_function(node:clang.cindex.Cursor):
    src = src_list()
    param_name = camel_to_snake(node.spelling);
    src += f"constexpr std::string_view to_string({node.spelling} {param_name}) noexcept {{"
    with src.indent():
        src += f"switch ({param_name}) {{"
        enum_names = list_enum_names(node)
        with src.indent():
            processed_values = []
            for n in enum_names:
                if n.value in processed_values: continue
                if n.name.endswith('_MAX_ENUM'): continue
                processed_values.append(n.value)
                
                src += f"case {n.name}:"
                with src.indent():
                    src += f"return \"{n.name}\";"
        src += "}"
        src += f"return \"Unknown {node.spelling}\";"
    src += "}"
    
    return src.get()

print(make_to_string_function(vk_enums['VkResult']))
print(make_to_string_function(vk_enums['VkImageTiling']))

constexpr std::string_view to_string(VkResult vk_result) noexcept {
    switch (vk_result) {
        case VK_SUCCESS:
            return "VK_SUCCESS";
        case VK_NOT_READY:
            return "VK_NOT_READY";
        case VK_TIMEOUT:
            return "VK_TIMEOUT";
        case VK_EVENT_SET:
            return "VK_EVENT_SET";
        case VK_EVENT_RESET:
            return "VK_EVENT_RESET";
        case VK_INCOMPLETE:
            return "VK_INCOMPLETE";
        case VK_ERROR_OUT_OF_HOST_MEMORY:
            return "VK_ERROR_OUT_OF_HOST_MEMORY";
        case VK_ERROR_OUT_OF_DEVICE_MEMORY:
            return "VK_ERROR_OUT_OF_DEVICE_MEMORY";
        case VK_ERROR_INITIALIZATION_FAILED:
            return "VK_ERROR_INITIALIZATION_FAILED";
        case VK_ERROR_DEVICE_LOST:
            return "VK_ERROR_DEVICE_LOST";
        case VK_ERROR_MEMORY_MAP_FAILED:
            return "VK_ERROR_MEMORY_MAP_FAILED";
        case VK_ERROR_LAYER_NOT_PRESENT:
            return "VK_ERROR_LAYER_NOT_PRE

In [ ]:
throw;
with open('Cory/include/Cory/vk/enum_utils.h', 'w') as f:
    f.write("""
#pragma once

#include <fmt/format.h>
#include <string_view>
#include <vulkan/vulkan.h>

namespace cory {
namespace vk {

""")
    
    for enum_node in vk_enums.values():
        f.write(make_to_string_function(enum_node))
    
    f.write("""
    
template <typename VulkanFlagBitsType, typename VulkanFlagsType>
std::string flag_bits_to_string(VulkanFlagsType flagBits)
{
        auto curFlag = (VulkanFlagBitsType)1;
    if (!flagBits) return "( )";

    std::string flagBitsString("( ");
    while (curFlag) {
        if (flagBits & curFlag) { flagBitsString += std::string(to_string((VulkanFlagBitsType)curFlag)) + " ";
        }
        curFlag = (VulkanFlagBitsType)(curFlag << 1);
    }
    return flagBitsString + ")";
}
    
} // namespace vk
} // namespace cory

template <> struct fmt::formatter<VkResult> {
    constexpr auto parse(format_parse_context &ctx)
    {
        auto it = ctx.begin(), end = ctx.end();
        if (it != end && *it != '}') throw format_error("invalid format");
        return ctx.end();
    }

    template <typename FormatContext, typename VulkanEnumType>
    auto format(const VulkanEnumType &e, FormatContext &ctx)
    {
        return format_to(ctx.out(), cory::vk::to_string(e));
    }
};

""")
    
    for ename in vk_enums.keys():
        if ename == "VkResult": continue
        f.write(f"template <> struct fmt::formatter<{ename}> : fmt::formatter<VkResult> {{}};\n")
    

# Command buffer

In [ ]:
cmdbuf_method_names = {k: camel_to_snake(k[5:]) for k in sorted(vk_funcs.keys()) if k.startswith("vkCmd")}

In [ ]:
func_node = vk_funcs['vkCmdBlitImage']

In [ ]:
def get_function_args(function_node:clang.cindex.Cursor):
    return [(a.type.spelling, a.spelling, a) for a in function_node.get_arguments()]

In [ ]:
get_function_args(func_node)

In [ ]:
src = src_list()
src >> 4
for (vkfunc_name, method) in cmdbuf_method_names.items():
    vkfunc = vk_funcs[vkfunc_name]
    args = get_function_args(vkfunc)[1:]
    
    args_list = ", ".join([f"{t} {n}" for t,n,_ in args])
    arg_names_list = ", ".join([n for t,n,_ in args])
    src += f"command_buffer& {method}({args_list}) {{"
    with src.indent():
        src += f"{vkfunc_name}(cmd_buffer_ptr_.get(), {arg_names_list});"
        src += f"return *this;"
    src += "}\n"
    
src << 4;

print(src.get())